In [ ]:
# Imports
import glob
import os
from PIL import Image

# For a progress bar
from tqdm import tqdm

import numpy as np
import pandas as pd
import scipy.signal

# Image processing tools
import skimage.feature
import skimage.filters
import skimage.filters.rank
import skimage.io
import skimage.morphology
import skimage.segmentation
from scipy import ndimage as ndi

import bebi103

import altair as alt

import altair_catplot as altcat

import bokeh
from bokeh.models.widgets import Panel, Tabs
from bokeh.io import output_file, show
from bokeh.plotting import figure

bokeh.io.output_notebook()

In [ ]:
def permutation_test(n, list_1, list_2):
    
    true_difference = np.abs(np.mean(list_1) - np.mean(list_2))

    combined_list = np.concatenate((list_1, list_2))

    # Initialize a list to store the results
    simulated_null_results = []

    # Choose the number of simulations
    n_simulations = n

    # Iterate 1000 simulations of the null hypothesis
    for _ in range(n_simulations):

        # Scramble the initial list
        sample = np.random.permutation(combined_list)

        # Draw samples the same length as the the initial samples from the permutated list 
        sample_a = sample[:len(list_1)]
        sample_b = sample[len(list_2):]

        # Calculate the difference between the mean beak depths of the generated samples 
        difference_of_means = np.abs(np.mean(sample_a) - np.mean(sample_b))

        # Store this information
        simulated_null_results.append(difference_of_means)

    count = 0

    # Count the number of simulations that have a test statistic as least as extreme as the experimental difference 
    for i in range(n_simulations):

        if simulated_null_results[i] >= true_difference:
            count += 1

    # Calculate the p value by dividing count by the number of simulations
    p_value = count/n_simulations

    return(p_value)

In [ ]:
def effect_size(list_1, list_2):
    mean_1 = np.mean(list_1)
    mean_2 = np.mean(list_2)
    true_difference = np.abs(mean_1 - mean_2)
    
    if mean_1 < mean_2:
        effect_size = mean_2/mean_1
    elif mean_1 > mean_2:
        effect_size = mean_1/mean_2
    else:
        effect_size = 0
        
    return effect_size

In [ ]:
def bootstrap(n_samples, sample_list):
    
    # Initialize lists to store the bootstrap samples
    bootstrap_samples = []

    # Initialize lists to store the bootstrap replicates for each year
    mean_list = []
    
    mean = np.mean(sample_list)

    for n in range(n_samples):

        # Randomly sample from the list the same number of measurements as the original sample
        sample = np.random.choice(sample_list, size=len(sample_list))

        # Determine the mean of the boot strap sample
        replicate = np.mean(sample)

        # Add the bootstrap samples to their respective list
        bootstrap_samples.append(sample)

        # Add the bootstrap replicates to their respective list
        mean_list.append(replicate)

    # Calculate a 95% confidence interval based on the sample
    confidence_interval = np.percentile(mean_list, [2.5, 97.5])
    
    return mean, confidence_interval

In [ ]:
def three_sample_statistics(list_1, list_2, list_3, labels, tissue, measure, n_bootstrap_samples = 10000, n_permutation_simulations = 10000):
    '''    For the hypothesis testing, the null hypothesis is that the transduction efficiency of capsid variants is derived from the same distribution. 
    The test statistic is the difference of means between the two samples. 
    Permutation samples are considered at least as extreme when the difference of means is greater than the actual difference of means.
    '''
    
    mean_1, ci_1 = bootstrap(n_bootstrap_samples, list_1)
    mean_2, ci_2 = bootstrap(n_bootstrap_samples, list_2)
    mean_3, ci_3 = bootstrap(n_bootstrap_samples, list_3)
    
    effect_1_2 = effect_size(list_1, list_2)
    effect_1_3 = effect_size(list_1, list_3)
    effect_2_3 = effect_size(list_2, list_3)
    
    p_val_1_2 = permutation_test(n_permutation_simulations, list_1, list_2)
    p_val_1_3 = permutation_test(n_permutation_simulations, list_1, list_3)
    p_val_2_3 = permutation_test(n_permutation_simulations, list_2, list_3)
    
    print("""
    Mean {19:s} {18:s} of {9:s}: {6:.2f}
    95% confidence interval of mean {19:s} {18:s} of {9:s}: [{0:.2f}, {1:.2f}]
    
    Mean {19:s} {18:s} of {10:s}: {7:.2f}
    95% confidence interval of mean {19:s} {18:s} of {10:s}: [{2:.2f}, {3:.2f}]
    
    Mean {19:s} {18:s} of {11:s}: {8:.2f}
    95% confidence interval of mean {19:s} {18:s} of {11:s}: [{4:.2f}, {5:.2f}]
    
    The effect size between {9:s} and {10:s} is {12:.1f} with a p-value of {15:.4f}
    The effect size between {9:s} and {11:s} is {13:.1f} with a p-value of {16:.4f}
    The effect size between {10:s} and {11:s} is {14:.1f} with a p-value of {17:.4f}
    """.format(*(tuple(ci_1) + tuple(ci_2) + tuple(ci_3)), 
               mean_1, 
               mean_2, 
               mean_3, 
               labels[0], 
               labels[1], 
               labels[2], 
               effect_1_2, 
               effect_1_3, 
               effect_2_3, 
               p_val_1_2, 
               p_val_1_3, 
               p_val_2_3,
               measure,
               tissue))

In [ ]:
df_lung = pd.read_csv('Lung_Paper_Images/Post-Quantification/Lung/20190912_Stitched_Lung_Cell_Counts_with_Brightness_sampled.csv', comment='#')

In [ ]:
df_lung['Tissue Area'] = [34703373,37375012,54529382,48297534,
                          29930164,26502408,35838440,34559984,
                          41851696,41270905,44916847,44317422,
                          39264712,41136449,49806081,48096510,
                          56377075,52059462,46826122,43021002,
                          44485649,40290462,43361463,37242966,
                          40996888,39114702,46437032,43174352,
                          53294502,57188550,36657925,36207557,
                          32240889,31383431,37814176,35841074]

df_lung['Cells Transduced per mm2'] = df_lung['Count'] / df_lung['Tissue Area'] * 1000000

df_lung['Virus'] = df_lung['Virus'].replace('CAPA4','AAV.CAP-A4')

In [ ]:
import json

for inds in np.arange(0, len(df_lung)): 
    df_lung.loc[inds, 'Brightness Median'] = np.median(json.loads(df_lung.loc[inds, 'Brightness List']))

grouped = df_lung.groupby(['Date','Virus','Animal'])
df_lung_counts = grouped['Cells Transduced per mm2'].mean().reset_index()
df_lung_brightness = grouped['Brightness Median'].mean().reset_index()

In [ ]:
# Add jitter; note kwarg p=p to add to existing figure
p = bebi103.viz.jitter(data=df_lung_counts, #.loc[inds, :], 
                       cats='Virus',
                       val='Cells Transduced per mm2',
                       y_axis_label='Transduction (cells/mm\u00b2)',
                       x_axis_label='Capsid',
                       size=5,
                       width = 0.4)

i = 0

bar_list = [0.5, 1.5, 2.5]
mean_width = 0.25
whisker_width = 0.05

for virus in ['AAV.CAP-A4', 'AAV5', 'AAV9']:
    inds = df_lung_counts['Virus'] == virus
    
    p.hbar(np.mean(df_lung_counts.loc[inds,'Cells Transduced per mm2']), height = 0, right=(bar_list[i]-mean_width), left=(bar_list[i]+mean_width), color = 'black')
    p.hbar(np.std(df_lung_counts.loc[inds,'Cells Transduced per mm2']) + np.mean(df_lung_counts.loc[inds,'Cells Transduced per mm2']), height = 0, right=(bar_list[i]-whisker_width), left=(bar_list[i]+whisker_width), color = 'black')
    p.hbar(np.mean(df_lung_counts.loc[inds,'Cells Transduced per mm2']) - np.std(df_lung_counts.loc[inds,'Cells Transduced per mm2']), height = 0, right=(bar_list[i]-whisker_width), left=(bar_list[i]+whisker_width), color = 'black')
    p.segment(bar_list[i], np.mean(df_lung_counts.loc[inds,'Cells Transduced per mm2']), bar_list[i], np.std(df_lung_counts.loc[inds,'Cells Transduced per mm2']) + np.mean(df_lung_counts.loc[inds,'Cells Transduced per mm2']) , color = 'black')
    p.segment(bar_list[i], np.mean(df_lung_counts.loc[inds,'Cells Transduced per mm2']), bar_list[i], np.mean(df_lung_counts.loc[inds,'Cells Transduced per mm2']) - np.std(df_lung_counts.loc[inds,'Cells Transduced per mm2']), color = 'black')
    
    i += 1
    
p.plot_height = 350
p.plot_width = 350

p.xaxis.major_label_text_font_size = '12pt'
p.yaxis.major_label_text_font_size = '12pt'

p.xaxis.axis_label_text_font_size = '18pt'
p.yaxis.axis_label_text_font_size = '18pt'

bokeh.io.show(p)

In [ ]:
# Add jitter; note kwarg p=p to add to existing figure
p = bebi103.viz.jitter(data=df_lung_brightness,
                       cats='Virus',
                       val='Brightness Median',
                       y_axis_label='Median Cell Brightness',
                       x_axis_label='Capsid',
                       size=5,
                       width = 0.4)

i = 0

bar_list = [0.5, 1.5, 2.5]
mean_width = 0.25
whisker_width = 0.05

for virus in ['AAV.CAP-A4', 'AAV5', 'AAV9']:
    inds = df_lung_brightness['Virus'] == virus
    
    p.hbar(np.mean(df_lung_brightness.loc[inds,'Brightness Median']), height = 0, right=(bar_list[i]-mean_width), left=(bar_list[i]+mean_width), color = 'black')
    p.hbar(np.std(df_lung_brightness.loc[inds,'Brightness Median']) + np.mean(df_lung_brightness.loc[inds,'Brightness Median']), height = 0, right=(bar_list[i]-whisker_width), left=(bar_list[i]+whisker_width), color = 'black')
    p.hbar(np.mean(df_lung_brightness.loc[inds,'Brightness Median']) - np.std(df_lung_brightness.loc[inds,'Brightness Median']), height = 0, right=(bar_list[i]-whisker_width), left=(bar_list[i]+whisker_width), color = 'black')
    p.segment(bar_list[i], np.mean(df_lung_brightness.loc[inds,'Brightness Median']), bar_list[i], np.std(df_lung_brightness.loc[inds,'Brightness Median']) + np.mean(df_lung_brightness.loc[inds,'Brightness Median']) , color = 'black')
    p.segment(bar_list[i], np.mean(df_lung_brightness.loc[inds,'Brightness Median']), bar_list[i], np.mean(df_lung_brightness.loc[inds,'Brightness Median']) - np.std(df_lung_brightness.loc[inds,'Brightness Median']), color = 'black')
    
    i += 1
    
p.plot_height = 350
p.plot_width = 350

p.xaxis.major_label_text_font_size = '12pt'
p.yaxis.major_label_text_font_size = '12pt'

p.xaxis.axis_label_text_font_size = '18pt'
p.yaxis.axis_label_text_font_size = '18pt'

bokeh.io.show(p)

In [ ]:
df_lung_cat = pd.read_csv('Lung_Paper_Images/Post-Quantification/Lung/20190822_Lung_Count_Catagorization.csv', comment='#')
df_lung_cat['Virus'] = df_lung_cat['Virus'].replace('CAPA4','AAV.CAP-A4')

In [ ]:
df_lung_cat['Total'] = df_lung_cat['Correct'] + df_lung_cat['False_Positive'] + df_lung_cat['False_Negative']
df_lung_cat['Accuracy'] = df_lung_cat['Correct']/df_lung_cat['Total']

In [ ]:
# Add jitter; note kwarg p=p to add to existing figure
p = bebi103.viz.jitter(data=df_lung_cat, #.loc[inds, :], 
                       cats='Virus',
                       val='Accuracy',
                       y_axis_label='Counting Accuracy',
                       x_axis_label='Capsid')

bar_list = [0.5, 1.5, 2.5]
mean_width = 0.25
i = 0
for virus in ['AAV.CAP-A4', 'AAV5', 'AAV9']:
    inds = df_lung_cat['Virus'] == virus
    p.hbar(np.mean(df_lung_cat.loc[inds,'Accuracy']), 
           height = 0, 
           right=(bar_list[i]-mean_width), 
           left=(bar_list[i]+mean_width), 
           color = 'black')
    i+=1

p.plot_height = 350
p.plot_width = 350

p.xaxis.major_label_text_font_size = '12pt'
p.yaxis.major_label_text_font_size = '12pt'

p.xaxis.axis_label_text_font_size = '18pt'
p.yaxis.axis_label_text_font_size = '18pt'

bokeh.io.show(p)

In [ ]:
# Initial Difference of means
inds = df_lung_counts['Virus'] == 'AAV.CAP-A4'
CAPA4_lung_transduction = df_lung_counts.loc[inds, 'Cells Transduced per mm2']
CAPA4_lung_brightness = df_lung_brightness.loc[inds, 'Brightness Median']

inds = df_lung_counts['Virus'] == 'AAV9'
AAV9_lung_transduction = df_lung_counts.loc[inds, 'Cells Transduced per mm2']
AAV9_lung_brightness = df_lung_brightness.loc[inds, 'Brightness Median']

inds = df_lung_counts['Virus'] == 'AAV5'
AAV5_lung_transduction = df_lung_counts.loc[inds, 'Cells Transduced per mm2']
AAV5_lung_brightness = df_lung_brightness.loc[inds, 'Brightness Median']

three_sample_statistics(CAPA4_lung_transduction, 
                        AAV9_lung_transduction, 
                        AAV5_lung_transduction, 
                        labels = ['AAV-CAP.A4', 'AAV9', 'AAV5'], 
                        tissue = 'lung', 
                        measure = 'transduction (cells/mm\u00b2)')

three_sample_statistics(CAPA4_lung_brightness, 
                        AAV9_lung_brightness, 
                        AAV5_lung_brightness, 
                        labels = ['AAV-CAP.A4', 'AAV9', 'AAV5'], 
                        tissue = 'lung', 
                        measure = 'cell brightness')

In [ ]:
df_liver = pd.read_csv('Lung_Paper_Images/Post-Quantification/Liver/20190912_Stitched_Liver_Cell_Counts_with_Brightness_sampled.csv', comment='#')

In [ ]:
df_liver['Tissue Area'] = [45020467, 41909715, 31720632, 30548805,
                           29718024, 34777635, 35961208, 36990765,
                           39156978, 37988034, 35722607, 37633761,
                           34242929, 35841776, 38771900, 48629552,
                           32462105, 33298795, 34652042, 32076311,
                           22396817, 22774738, 32894428, 34784889,
                           28000278, 35160915, 32174737, 32325632,
                           44929928, 45981690, 36069262, 36358838,
                           46382678, 36875011, 33254993, 35025299]

df_liver['Cells Transduced per mm2'] = df_liver['Count'] / df_liver['Tissue Area'] * 1000000

df_liver['Virus'] = df_liver['Virus'].replace('CAPA4','AAV.CAP-A4')

In [ ]:
for inds in np.arange(0, len(df_liver)): 
    df_liver.loc[inds, 'Brightness Median'] = np.median(json.loads(df_liver.loc[inds, 'Brightness List']))

grouped = df_liver.groupby(['Date','Virus','Animal'])
df_liver_counts = grouped['Cells Transduced per mm2'].mean().reset_index()
df_liver_brightness = grouped['Brightness Median'].mean().reset_index()

In [ ]:
# Add jitter; note kwarg p=p to add to existing figure
p = bebi103.viz.jitter(data=df_liver_counts, #.loc[inds, :], 
                       cats='Virus',
                       val='Cells Transduced per mm2',
                       y_axis_label='Transduction (cells/mm\u00b2)',
                       x_axis_label='Capsid',
                       size=5,
                       width = 0.4)

i = 0

bar_list = [0.5, 1.5, 2.5]
mean_width = 0.25
whisker_width = 0.05

for virus in ['AAV.CAP-A4', 'AAV5', 'AAV9']:
    inds = df_liver_counts['Virus'] == virus
    
    p.hbar(np.mean(df_liver_counts.loc[inds,'Cells Transduced per mm2']), height = 0, right=(bar_list[i]-mean_width), left=(bar_list[i]+mean_width), color = 'black')
    p.hbar(np.std(df_liver_counts.loc[inds,'Cells Transduced per mm2']) + np.mean(df_liver_counts.loc[inds,'Cells Transduced per mm2']), height = 0, right=(bar_list[i]-whisker_width), left=(bar_list[i]+whisker_width), color = 'black')
    p.hbar(np.mean(df_liver_counts.loc[inds,'Cells Transduced per mm2']) - np.std(df_liver_counts.loc[inds,'Cells Transduced per mm2']), height = 0, right=(bar_list[i]-whisker_width), left=(bar_list[i]+whisker_width), color = 'black')
    p.segment(bar_list[i], np.mean(df_liver_counts.loc[inds,'Cells Transduced per mm2']), bar_list[i], np.std(df_liver_counts.loc[inds,'Cells Transduced per mm2']) + np.mean(df_liver_counts.loc[inds,'Cells Transduced per mm2']) , color = 'black')
    p.segment(bar_list[i], np.mean(df_liver_counts.loc[inds,'Cells Transduced per mm2']), bar_list[i], np.mean(df_liver_counts.loc[inds,'Cells Transduced per mm2']) - np.std(df_liver_counts.loc[inds,'Cells Transduced per mm2']), color = 'black')
    
    i += 1
    
p.plot_height = 350
p.plot_width = 350

p.xaxis.major_label_text_font_size = '12pt'
p.yaxis.major_label_text_font_size = '12pt'

p.xaxis.axis_label_text_font_size = '18pt'
p.yaxis.axis_label_text_font_size = '18pt'

bokeh.io.show(p)

In [ ]:
# Add jitter; note kwarg p=p to add to existing figure
p = bebi103.viz.jitter(data=df_liver_brightness,
                       cats='Virus',
                       val='Brightness Median',
                       y_axis_label='Median Cell Brightness',
                       x_axis_label='Capsid',
                       size=5,
                       width = 0.4)

i = 0

bar_list = [0.5, 1.5, 2.5]
mean_width = 0.25
whisker_width = 0.05

for virus in ['AAV.CAP-A4', 'AAV5', 'AAV9']:
    inds = df_liver_brightness['Virus'] == virus
    
    p.hbar(np.mean(df_liver_brightness.loc[inds,'Brightness Median']), height = 0, right=(bar_list[i]-mean_width), left=(bar_list[i]+mean_width), color = 'black')
    p.hbar(np.std(df_liver_brightness.loc[inds,'Brightness Median']) + np.mean(df_liver_brightness.loc[inds,'Brightness Median']), height = 0, right=(bar_list[i]-whisker_width), left=(bar_list[i]+whisker_width), color = 'black')
    p.hbar(np.mean(df_liver_brightness.loc[inds,'Brightness Median']) - np.std(df_liver_brightness.loc[inds,'Brightness Median']), height = 0, right=(bar_list[i]-whisker_width), left=(bar_list[i]+whisker_width), color = 'black')
    p.segment(bar_list[i], np.mean(df_liver_brightness.loc[inds,'Brightness Median']), bar_list[i], np.std(df_liver_brightness.loc[inds,'Brightness Median']) + np.mean(df_liver_brightness.loc[inds,'Brightness Median']) , color = 'black')
    p.segment(bar_list[i], np.mean(df_liver_brightness.loc[inds,'Brightness Median']), bar_list[i], np.mean(df_liver_brightness.loc[inds,'Brightness Median']) - np.std(df_liver_brightness.loc[inds,'Brightness Median']), color = 'black')
    
    i += 1
    
p.plot_height = 350
p.plot_width = 350

p.xaxis.major_label_text_font_size = '12pt'
p.yaxis.major_label_text_font_size = '12pt'

p.xaxis.axis_label_text_font_size = '18pt'
p.yaxis.axis_label_text_font_size = '18pt'

bokeh.io.show(p)

In [ ]:
df_liver_cat = pd.read_csv('Lung_Paper_Images/Post-Quantification/Liver/20190822_Liver_Count_Catagorization.csv', comment='#')
df_liver_cat['Virus'] = df_liver_cat['Virus'].replace('CAPA4','AAV.CAP-A4')
df_liver_cat['Total'] = df_liver_cat['Correct'] + df_liver_cat['False_Positive'] + df_liver_cat['False_Negative']
df_liver_cat['Accuracy'] = df_liver_cat['Correct']/df_liver_cat['Total']

In [ ]:
# Add jitter; note kwarg p=p to add to existing figure
p = bebi103.viz.jitter(data=df_liver_cat, #.loc[inds, :], 
                       cats='Virus',
                       val='Accuracy',
                       y_axis_label='Counting Accuracy',
                       x_axis_label='Capsid')

bar_list = [0.5, 1.5, 2.5]
mean_width = 0.25
i = 0
for virus in ['AAV.CAP-A4', 'AAV5', 'AAV9']:
    inds = df_liver_cat['Virus'] == virus
    p.hbar(np.mean(df_liver_cat.loc[inds,'Accuracy']), 
           height = 0.0005, 
           right=(bar_list[i]-mean_width), 
           left=(bar_list[i]+mean_width), 
           color = 'black')
    i+=1

p.plot_height = 350
p.plot_width = 350

p.xaxis.major_label_text_font_size = '12pt'
p.yaxis.major_label_text_font_size = '12pt'

p.xaxis.axis_label_text_font_size = '18pt'
p.yaxis.axis_label_text_font_size = '18pt'

bokeh.io.show(p)

In [ ]:
# Initial Difference of means
inds = df_liver_counts['Virus'] == 'AAV.CAP-A4'
CAPA4_liver_transduction = df_liver_counts.loc[inds, 'Cells Transduced per mm2']
CAPA4_liver_brightness = df_liver_brightness.loc[inds, 'Brightness Median']

inds = df_liver_counts['Virus'] == 'AAV9'
AAV9_liver_transduction = df_liver_counts.loc[inds, 'Cells Transduced per mm2']
AAV9_liver_brightness = df_liver_brightness.loc[inds, 'Brightness Median']

inds = df_liver_counts['Virus'] == 'AAV5'
AAV5_liver_transduction = df_liver_counts.loc[inds, 'Cells Transduced per mm2']
AAV5_liver_brightness = df_liver_brightness.loc[inds, 'Brightness Median']

three_sample_statistics(CAPA4_liver_transduction, 
                        AAV9_liver_transduction, 
                        AAV5_liver_transduction, 
                        labels = ['AAV-CAP.A4', 'AAV9', 'AAV5'], 
                        tissue = 'liver', 
                        measure = 'transduction (cells/mm\u00b2)')

three_sample_statistics(CAPA4_liver_brightness, 
                        AAV9_liver_brightness, 
                        AAV5_liver_brightness, 
                        labels = ['AAV-CAP.A4', 'AAV9', 'AAV5'], 
                        tissue = 'liver', 
                        measure = 'cell brightness')

In [ ]:
df_brain = pd.read_csv('Lung_Paper_Images/Post-Quantification/Brain/20190917_Stitched_Brain_Cell_Counts_with_Brightness_sampled.csv', comment='#')

In [ ]:
df_brain['Tissue Area'] = [74230403,74109826,73869403,72333530,
                           76013061,76528594,79420118,80623852,
                           71042690,71858814,73840606,76350292,
                           75461292,75172493,76427048,76455846,
                           68833492,68529611,75873963,70325503,
                           76333124,74612312,75258519,70243752, 
                           80744905, 82381853, 82266008, 86426562, 
                           82346451, 75747026, 73617495, 75073311,
                           82025888, 78641641, 74563719, 80724917]

df_brain['Cells Transduced per mm2'] = df_brain['Count'] / df_brain['Tissue Area'] * 1000000
df_brain['Virus'] = df_brain['Virus'].replace('CAPA4','AAV.CAP-A4')

In [ ]:
for inds in np.arange(0, len(df_brain)): 
    df_brain.loc[inds, 'Brightness Median'] = np.median(json.loads(df_brain.loc[inds, 'Brightness List']))

grouped = df_brain.groupby(['Date','Virus','Animal'])
df_brain_counts = grouped['Cells Transduced per mm2'].mean().reset_index()
df_brain_brightness = grouped['Brightness Median'].mean().reset_index()

In [ ]:
# Add jitter; note kwarg p=p to add to existing figure
p = bebi103.viz.jitter(data=df_brain_counts, #.loc[inds, :], 
                       cats='Virus',
                       val='Cells Transduced per mm2',
                       y_axis_label='Transduction (cells/mm\u00b2)',
                       x_axis_label='Capsid',
                       size=5,
                       width = 0.4)

i = 0

bar_list = [0.5, 1.5, 2.5]
mean_width = 0.25
whisker_width = 0.05

for virus in ['AAV.CAP-A4', 'AAV5', 'AAV9']:
    inds = df_brain_counts['Virus'] == virus
    
    p.hbar(np.mean(df_brain_counts.loc[inds,'Cells Transduced per mm2']), height = 0, right=(bar_list[i]-mean_width), left=(bar_list[i]+mean_width), color = 'black')
    p.hbar(np.std(df_brain_counts.loc[inds,'Cells Transduced per mm2']) + np.mean(df_brain_counts.loc[inds,'Cells Transduced per mm2']), height = 0, right=(bar_list[i]-whisker_width), left=(bar_list[i]+whisker_width), color = 'black')
    p.hbar(np.mean(df_brain_counts.loc[inds,'Cells Transduced per mm2']) - np.std(df_brain_counts.loc[inds,'Cells Transduced per mm2']), height = 0, right=(bar_list[i]-whisker_width), left=(bar_list[i]+whisker_width), color = 'black')
    p.segment(bar_list[i], np.mean(df_brain_counts.loc[inds,'Cells Transduced per mm2']), bar_list[i], np.std(df_brain_counts.loc[inds,'Cells Transduced per mm2']) + np.mean(df_brain_counts.loc[inds,'Cells Transduced per mm2']) , color = 'black')
    p.segment(bar_list[i], np.mean(df_brain_counts.loc[inds,'Cells Transduced per mm2']), bar_list[i], np.mean(df_brain_counts.loc[inds,'Cells Transduced per mm2']) - np.std(df_brain_counts.loc[inds,'Cells Transduced per mm2']), color = 'black')
    
    i += 1
    
p.plot_height = 350
p.plot_width = 350

p.xaxis.major_label_text_font_size = '12pt'
p.yaxis.major_label_text_font_size = '12pt'

p.xaxis.axis_label_text_font_size = '18pt'
p.yaxis.axis_label_text_font_size = '18pt'

bokeh.io.show(p)

In [ ]:
# Add jitter; note kwarg p=p to add to existing figure
p = bebi103.viz.jitter(data=df_brain_brightness,
                       cats='Virus',
                       val='Brightness Median',
                       y_axis_label='Median Cell Brightness',
                       x_axis_label='Capsid',
                       size=5,
                       width = 0.4)

i = 0

bar_list = [0.5, 1.5, 2.5]
mean_width = 0.25
whisker_width = 0.05

for virus in ['AAV.CAP-A4', 'AAV5', 'AAV9']:
    inds = df_brain_brightness['Virus'] == virus
    
    p.hbar(np.mean(df_brain_brightness.loc[inds,'Brightness Median']), height = 0, right=(bar_list[i]-mean_width), left=(bar_list[i]+mean_width), color = 'black')
    p.hbar(np.std(df_brain_brightness.loc[inds,'Brightness Median']) + np.mean(df_brain_brightness.loc[inds,'Brightness Median']), height = 0, right=(bar_list[i]-whisker_width), left=(bar_list[i]+whisker_width), color = 'black')
    p.hbar(np.mean(df_brain_brightness.loc[inds,'Brightness Median']) - np.std(df_brain_brightness.loc[inds,'Brightness Median']), height = 0, right=(bar_list[i]-whisker_width), left=(bar_list[i]+whisker_width), color = 'black')
    p.segment(bar_list[i], np.mean(df_brain_brightness.loc[inds,'Brightness Median']), bar_list[i], np.std(df_brain_brightness.loc[inds,'Brightness Median']) + np.mean(df_brain_brightness.loc[inds,'Brightness Median']) , color = 'black')
    p.segment(bar_list[i], np.mean(df_brain_brightness.loc[inds,'Brightness Median']), bar_list[i], np.mean(df_brain_brightness.loc[inds,'Brightness Median']) - np.std(df_brain_brightness.loc[inds,'Brightness Median']), color = 'black')
    
    i += 1
    
p.plot_height = 350
p.plot_width = 350

p.xaxis.major_label_text_font_size = '12pt'
p.yaxis.major_label_text_font_size = '12pt'

p.xaxis.axis_label_text_font_size = '18pt'
p.yaxis.axis_label_text_font_size = '18pt'

bokeh.io.show(p)

In [ ]:
# Initial Difference of means
inds = df_brain_counts['Virus'] == 'AAV.CAP-A4'
CAPA4_brain_transduction = df_brain_counts.loc[inds, 'Cells Transduced per mm2']
CAPA4_brain_brightness = df_brain_brightness.loc[inds, 'Brightness Median']

inds = df_brain_counts['Virus'] == 'AAV9'
AAV9_brain_transduction = df_brain_counts.loc[inds, 'Cells Transduced per mm2']
AAV9_brain_brightness = df_brain_brightness.loc[inds, 'Brightness Median']

inds = df_brain_counts['Virus'] == 'AAV5'
AAV5_brain_transduction = df_brain_counts.loc[inds, 'Cells Transduced per mm2']
AAV5_brain_brightness = df_brain_brightness.loc[inds, 'Brightness Median']

three_sample_statistics(CAPA4_brain_transduction, 
                        AAV9_brain_transduction, 
                        AAV5_brain_transduction, 
                        labels = ['AAV-CAP.A4', 'AAV9', 'AAV5'], 
                        tissue = 'brain', 
                        measure = 'transduction (cells/mm\u00b2)')

three_sample_statistics(CAPA4_brain_brightness, 
                        AAV9_brain_brightness, 
                        AAV5_brain_brightness, 
                        labels = ['AAV-CAP.A4', 'AAV9', 'AAV5'], 
                        tissue = 'brain', 
                        measure = 'cell brightness')

In [ ]:
df_kidney = pd.read_csv('Lung_Paper_Images/Post-Quantification/Kidney/20190913_Stitched_Kidney_Cell_Counts_with_Brightness_sampled.csv', comment='#')

In [ ]:
df_kidney['Tissue Area'] = [23840087,23248910,23695820,25325666,
                            21315929,23092826,22445314,22399548,
                            22164351,22334742,21140960,21325033,
                            21953236,23880419,22995086,23758651,
                            19911624,20889529,20630667,20015154,
                            28048157,28176264,19522765,19959024,
                            28563078,30958516,23372458,31694510,
                            17884701,19923352,22188833,23494811,
                            17328792,18533231,24826537,24731486]

df_kidney['Cells Transduced per mm2'] = df_kidney['Count'] / df_kidney['Tissue Area'] * 1000000
df_kidney['Virus'] = df_kidney['Virus'].replace('CAPA4','AAV.CAP-A4')

In [ ]:
for inds in np.arange(0, len(df_kidney)): 
    df_kidney.loc[inds, 'Brightness Median'] = np.median(json.loads(df_kidney.loc[inds, 'Brightness List']))

grouped = df_kidney.groupby(['Date','Virus','Animal'])
df_kidney_counts = grouped['Cells Transduced per mm2'].mean().reset_index()
df_kidney_brightness = grouped['Brightness Median'].mean().reset_index()

In [ ]:
# Add jitter; note kwarg p=p to add to existing figure
p = bebi103.viz.jitter(data=df_kidney_counts, #.loc[inds, :], 
                       cats='Virus',
                       val='Cells Transduced per mm2',
                       y_axis_label='Transduction (cells/mm\u00b2)',
                       x_axis_label='Capsid',
                       size=5,
                       width = 0.4)

i = 0

bar_list = [0.5, 1.5, 2.5]
mean_width = 0.25
whisker_width = 0.05

for virus in ['AAV.CAP-A4', 'AAV5', 'AAV9']:
    inds = df_kidney_counts['Virus'] == virus
    
    p.hbar(np.mean(df_kidney_counts.loc[inds,'Cells Transduced per mm2']), height = 0, right=(bar_list[i]-mean_width), left=(bar_list[i]+mean_width), color = 'black')
    p.hbar(np.std(df_kidney_counts.loc[inds,'Cells Transduced per mm2']) + np.mean(df_kidney_counts.loc[inds,'Cells Transduced per mm2']), height = 0, right=(bar_list[i]-whisker_width), left=(bar_list[i]+whisker_width), color = 'black')
    p.hbar(np.mean(df_kidney_counts.loc[inds,'Cells Transduced per mm2']) - np.std(df_kidney_counts.loc[inds,'Cells Transduced per mm2']), height = 0, right=(bar_list[i]-whisker_width), left=(bar_list[i]+whisker_width), color = 'black')
    p.segment(bar_list[i], np.mean(df_kidney_counts.loc[inds,'Cells Transduced per mm2']), bar_list[i], np.std(df_kidney_counts.loc[inds,'Cells Transduced per mm2']) + np.mean(df_kidney_counts.loc[inds,'Cells Transduced per mm2']) , color = 'black')
    p.segment(bar_list[i], np.mean(df_kidney_counts.loc[inds,'Cells Transduced per mm2']), bar_list[i], np.mean(df_kidney_counts.loc[inds,'Cells Transduced per mm2']) - np.std(df_kidney_counts.loc[inds,'Cells Transduced per mm2']), color = 'black')
    
    i += 1
    
p.plot_height = 350
p.plot_width = 350

p.xaxis.major_label_text_font_size = '12pt'
p.yaxis.major_label_text_font_size = '12pt'

p.xaxis.axis_label_text_font_size = '18pt'
p.yaxis.axis_label_text_font_size = '18pt'

bokeh.io.show(p)

In [ ]:
# Add jitter; note kwarg p=p to add to existing figure
p = bebi103.viz.jitter(data=df_kidney_brightness,
                       cats='Virus',
                       val='Brightness Median',
                       y_axis_label='Median Cell Brightness',
                       x_axis_label='Capsid',
                       size=5,
                       width = 0.4)

i = 0

bar_list = [0.5, 1.5, 2.5]
mean_width = 0.25
whisker_width = 0.05

for virus in ['AAV.CAP-A4', 'AAV5', 'AAV9']:
    inds = df_kidney_brightness['Virus'] == virus
    
    p.hbar(np.mean(df_kidney_brightness.loc[inds,'Brightness Median']), height = 0, right=(bar_list[i]-mean_width), left=(bar_list[i]+mean_width), color = 'black')
    p.hbar(np.std(df_kidney_brightness.loc[inds,'Brightness Median']) + np.mean(df_kidney_brightness.loc[inds,'Brightness Median']), height = 0, right=(bar_list[i]-whisker_width), left=(bar_list[i]+whisker_width), color = 'black')
    p.hbar(np.mean(df_kidney_brightness.loc[inds,'Brightness Median']) - np.std(df_kidney_brightness.loc[inds,'Brightness Median']), height = 0, right=(bar_list[i]-whisker_width), left=(bar_list[i]+whisker_width), color = 'black')
    p.segment(bar_list[i], np.mean(df_kidney_brightness.loc[inds,'Brightness Median']), bar_list[i], np.std(df_kidney_brightness.loc[inds,'Brightness Median']) + np.mean(df_kidney_brightness.loc[inds,'Brightness Median']) , color = 'black')
    p.segment(bar_list[i], np.mean(df_kidney_brightness.loc[inds,'Brightness Median']), bar_list[i], np.mean(df_kidney_brightness.loc[inds,'Brightness Median']) - np.std(df_kidney_brightness.loc[inds,'Brightness Median']), color = 'black')
    
    i += 1
    
p.plot_height = 350
p.plot_width = 350

p.xaxis.major_label_text_font_size = '12pt'
p.yaxis.major_label_text_font_size = '12pt'

p.xaxis.axis_label_text_font_size = '18pt'
p.yaxis.axis_label_text_font_size = '18pt'

bokeh.io.show(p)

In [ ]:
# Initial Difference of means
inds = df_kidney_counts['Virus'] == 'AAV.CAP-A4'
CAPA4_kidney_transduction = df_kidney_counts.loc[inds, 'Cells Transduced per mm2']
CAPA4_kidney_brightness = df_kidney_brightness.loc[inds, 'Brightness Median']

inds = df_kidney_counts['Virus'] == 'AAV9'
AAV9_kidney_transduction = df_kidney_counts.loc[inds, 'Cells Transduced per mm2']
AAV9_kidney_brightness = df_kidney_brightness.loc[inds, 'Brightness Median']

inds = df_kidney_counts['Virus'] == 'AAV5'
AAV5_kidney_transduction = df_kidney_counts.loc[inds, 'Cells Transduced per mm2']
AAV5_kidney_brightness = df_kidney_brightness.loc[inds, 'Brightness Median']

three_sample_statistics(CAPA4_kidney_transduction, 
                        AAV9_kidney_transduction, 
                        AAV5_kidney_transduction, 
                        labels = ['AAV-CAP.A4', 'AAV9', 'AAV5'], 
                        tissue = 'kidney', 
                        measure = 'transduction (cells/mm\u00b2)')

three_sample_statistics(CAPA4_kidney_brightness, 
                        AAV9_kidney_brightness, 
                        AAV5_kidney_brightness, 
                        labels = ['AAV-CAP.A4', 'AAV9', 'AAV5'], 
                        tissue = 'kidney', 
                        measure = 'cell brightness')

In [ ]:
df_spleen = pd.read_csv('Lung_Paper_Images/Post-Quantification/Spleen/20190913_Stitched_Spleen_Cell_Counts_with_Brightness_sampled.csv', comment='#')

In [ ]:
df_spleen['Tissue Area'] = [56038839, 57224979, 46704861, 48592679,
                             44920850, 47456692, 43795053, 48751981,
                             38264289, 42019557, 46598258, 51080694,
                             34973496, 38293002, 45294363, 50972229,
                             42943843, 46662193, 35518169, 39704012,
                             57072416, 66204086, 33623165, 41438040,
                             56828824, 51732420, 46538680, 47436817,
                             45541848, 45790972, 35764552, 33480571,
                             40243760, 38648107, 45736307, 38836106]

df_spleen['Cells Transduced per mm2'] = df_spleen['Count'] / df_spleen['Tissue Area'] * 1000000
df_spleen['Virus'] = df_spleen['Virus'].replace('CAPA4','AAV.CAP-A4')

In [ ]:
for inds in np.arange(0, len(df_spleen)): 
    df_spleen.loc[inds, 'Brightness Median'] = np.median(json.loads(df_spleen.loc[inds, 'Brightness List']))

grouped = df_spleen.groupby(['Date','Virus','Animal'])
df_spleen_counts = grouped['Cells Transduced per mm2'].mean().reset_index()
df_spleen_brightness = grouped['Brightness Median'].mean().reset_index()

In [ ]:
# Add jitter; note kwarg p=p to add to existing figure
p = bebi103.viz.jitter(data=df_spleen_counts, #.loc[inds, :], 
                       cats='Virus',
                       val='Cells Transduced per mm2',
                       y_axis_label='Transduction (cells/mm\u00b2)',
                       x_axis_label='Capsid',
                       size=5,
                       width = 0.4)

i = 0

bar_list = [0.5, 1.5, 2.5]
mean_width = 0.25
whisker_width = 0.05

for virus in ['AAV.CAP-A4', 'AAV5', 'AAV9']:
    inds = df_spleen_counts['Virus'] == virus
    
    p.hbar(np.mean(df_spleen_counts.loc[inds,'Cells Transduced per mm2']), height = 0, right=(bar_list[i]-mean_width), left=(bar_list[i]+mean_width), color = 'black')
    p.hbar(np.std(df_spleen_counts.loc[inds,'Cells Transduced per mm2']) + np.mean(df_spleen_counts.loc[inds,'Cells Transduced per mm2']), height = 0, right=(bar_list[i]-whisker_width), left=(bar_list[i]+whisker_width), color = 'black')
    p.hbar(np.mean(df_spleen_counts.loc[inds,'Cells Transduced per mm2']) - np.std(df_spleen_counts.loc[inds,'Cells Transduced per mm2']), height = 0, right=(bar_list[i]-whisker_width), left=(bar_list[i]+whisker_width), color = 'black')
    p.segment(bar_list[i], np.mean(df_spleen_counts.loc[inds,'Cells Transduced per mm2']), bar_list[i], np.std(df_spleen_counts.loc[inds,'Cells Transduced per mm2']) + np.mean(df_spleen_counts.loc[inds,'Cells Transduced per mm2']) , color = 'black')
    p.segment(bar_list[i], np.mean(df_spleen_counts.loc[inds,'Cells Transduced per mm2']), bar_list[i], np.mean(df_spleen_counts.loc[inds,'Cells Transduced per mm2']) - np.std(df_spleen_counts.loc[inds,'Cells Transduced per mm2']), color = 'black')
    
    i += 1
    
p.plot_height = 350
p.plot_width = 350

p.xaxis.major_label_text_font_size = '12pt'
p.yaxis.major_label_text_font_size = '12pt'

p.xaxis.axis_label_text_font_size = '18pt'
p.yaxis.axis_label_text_font_size = '18pt'

bokeh.io.show(p)

In [ ]:
# Add jitter; note kwarg p=p to add to existing figure
p = bebi103.viz.jitter(data=df_spleen_brightness,
                       cats='Virus',
                       val='Brightness Median',
                       y_axis_label='Median Cell Brightness',
                       x_axis_label='Capsid',
                       size=5,
                       width = 0.4)

i = 0

bar_list = [0.5, 1.5, 2.5]
mean_width = 0.25
whisker_width = 0.05

for virus in ['AAV.CAP-A4', 'AAV5', 'AAV9']:
    inds = df_spleen_brightness['Virus'] == virus
    
    p.hbar(np.mean(df_spleen_brightness.loc[inds,'Brightness Median']), height = 0, right=(bar_list[i]-mean_width), left=(bar_list[i]+mean_width), color = 'black')
    p.hbar(np.std(df_spleen_brightness.loc[inds,'Brightness Median']) + np.mean(df_spleen_brightness.loc[inds,'Brightness Median']), height = 0, right=(bar_list[i]-whisker_width), left=(bar_list[i]+whisker_width), color = 'black')
    p.hbar(np.mean(df_spleen_brightness.loc[inds,'Brightness Median']) - np.std(df_spleen_brightness.loc[inds,'Brightness Median']), height = 0, right=(bar_list[i]-whisker_width), left=(bar_list[i]+whisker_width), color = 'black')
    p.segment(bar_list[i], np.mean(df_spleen_brightness.loc[inds,'Brightness Median']), bar_list[i], np.std(df_spleen_brightness.loc[inds,'Brightness Median']) + np.mean(df_spleen_brightness.loc[inds,'Brightness Median']) , color = 'black')
    p.segment(bar_list[i], np.mean(df_spleen_brightness.loc[inds,'Brightness Median']), bar_list[i], np.mean(df_spleen_brightness.loc[inds,'Brightness Median']) - np.std(df_spleen_brightness.loc[inds,'Brightness Median']), color = 'black')
    
    i += 1
    
p.plot_height = 350
p.plot_width = 350

p.xaxis.major_label_text_font_size = '12pt'
p.yaxis.major_label_text_font_size = '12pt'

p.xaxis.axis_label_text_font_size = '18pt'
p.yaxis.axis_label_text_font_size = '18pt'

bokeh.io.show(p)

In [ ]:
# Initial Difference of means
inds = df_spleen_counts['Virus'] == 'AAV.CAP-A4'
CAPA4_spleen_transduction = df_spleen_counts.loc[inds, 'Cells Transduced per mm2']
CAPA4_spleen_brightness = df_spleen_brightness.loc[inds, 'Brightness Median']

inds = df_spleen_counts['Virus'] == 'AAV9'
AAV9_spleen_transduction = df_spleen_counts.loc[inds, 'Cells Transduced per mm2']
AAV9_spleen_brightness = df_spleen_brightness.loc[inds, 'Brightness Median']

inds = df_spleen_counts['Virus'] == 'AAV5'
AAV5_spleen_transduction = df_spleen_counts.loc[inds, 'Cells Transduced per mm2']
AAV5_spleen_brightness = df_spleen_brightness.loc[inds, 'Brightness Median']

three_sample_statistics(CAPA4_spleen_transduction, 
                        AAV9_spleen_transduction, 
                        AAV5_spleen_transduction, 
                        labels = ['AAV-CAP.A4', 'AAV9', 'AAV5'], 
                        tissue = 'spleen', 
                        measure = 'transduction (cells/mm\u00b2)')

three_sample_statistics(CAPA4_spleen_brightness, 
                        AAV9_spleen_brightness, 
                        AAV5_spleen_brightness, 
                        labels = ['AAV-CAP.A4', 'AAV9', 'AAV5'], 
                        tissue = 'spleen', 
                        measure = 'cell brightness')

In [ ]:
df_testes = pd.read_csv('Lung_Paper_Images/Post-Quantification/Testes/20190916_Stitched_Testes_Cell_Counts_with_Brightness_sampled.csv', comment='#')

In [ ]:
df_testes['Tissue Area'] = [22843816,25551689,15123764,18260951,
                            15235548,17199724,12503088,15316954,
                            15463481,19597953,12028821,15239262,
                            8023274,10821922,16740999,20425582,
                            16423927,17667787,17437276,18331587,
                            8238530,10073093,9547745,12191933,
                            33633524,33794052,21832925,27129648,
                            24526479,31519714,19874169,17578854,
                            23113836,31452010,20313380,29439704]

df_testes['Cells Transduced per mm2'] = df_testes['Count'] / df_testes['Tissue Area'] * 1000000
df_testes['Virus'] = df_testes['Virus'].replace('CAPA4','AAV.CAP-A4')

In [ ]:
for inds in np.arange(0, len(df_testes)): 
    df_testes.loc[inds, 'Brightness Median'] = np.median(json.loads(df_testes.loc[inds, 'Brightness List']))

grouped = df_testes.groupby(['Date','Virus','Animal'])
df_testes_counts = grouped['Cells Transduced per mm2'].mean().reset_index()
df_testes_brightness = grouped['Brightness Median'].mean().reset_index()

In [ ]:
# Add jitter; note kwarg p=p to add to existing figure
p = bebi103.viz.jitter(data=df_testes_counts, #.loc[inds, :], 
                       cats='Virus',
                       val='Cells Transduced per mm2',
                       y_axis_label='Transduction (cells/mm\u00b2)',
                       x_axis_label='Capsid',
                       size=5,
                       width = 0.4)

i = 0

bar_list = [0.5, 1.5, 2.5]
mean_width = 0.25
whisker_width = 0.05

for virus in ['AAV.CAP-A4', 'AAV5', 'AAV9']:
    inds = df_testes_counts['Virus'] == virus
    
    p.hbar(np.mean(df_testes_counts.loc[inds,'Cells Transduced per mm2']), height = 0, right=(bar_list[i]-mean_width), left=(bar_list[i]+mean_width), color = 'black')
    p.hbar(np.std(df_testes_counts.loc[inds,'Cells Transduced per mm2']) + np.mean(df_testes_counts.loc[inds,'Cells Transduced per mm2']), height = 0, right=(bar_list[i]-whisker_width), left=(bar_list[i]+whisker_width), color = 'black')
    p.hbar(np.mean(df_testes_counts.loc[inds,'Cells Transduced per mm2']) - np.std(df_testes_counts.loc[inds,'Cells Transduced per mm2']), height = 0, right=(bar_list[i]-whisker_width), left=(bar_list[i]+whisker_width), color = 'black')
    p.segment(bar_list[i], np.mean(df_testes_counts.loc[inds,'Cells Transduced per mm2']), bar_list[i], np.std(df_testes_counts.loc[inds,'Cells Transduced per mm2']) + np.mean(df_testes_counts.loc[inds,'Cells Transduced per mm2']) , color = 'black')
    p.segment(bar_list[i], np.mean(df_testes_counts.loc[inds,'Cells Transduced per mm2']), bar_list[i], np.mean(df_testes_counts.loc[inds,'Cells Transduced per mm2']) - np.std(df_testes_counts.loc[inds,'Cells Transduced per mm2']), color = 'black')
    
    i += 1
    
p.plot_height = 350
p.plot_width = 350

p.xaxis.major_label_text_font_size = '12pt'
p.yaxis.major_label_text_font_size = '12pt'

p.xaxis.axis_label_text_font_size = '18pt'
p.yaxis.axis_label_text_font_size = '18pt'

bokeh.io.show(p)

In [ ]:
# Add jitter; note kwarg p=p to add to existing figure
p = bebi103.viz.jitter(data=df_testes_brightness,
                       cats='Virus',
                       val='Brightness Median',
                       y_axis_label='Median Cell Brightness',
                       x_axis_label='Capsid',
                       size=5,
                       width = 0.4)

i = 0

bar_list = [0.5, 1.5, 2.5]
mean_width = 0.25
whisker_width = 0.05

for virus in ['AAV.CAP-A4', 'AAV5', 'AAV9']:
    inds = df_testes_brightness['Virus'] == virus
    
    p.hbar(np.mean(df_testes_brightness.loc[inds,'Brightness Median']), height = 0, right=(bar_list[i]-mean_width), left=(bar_list[i]+mean_width), color = 'black')
    p.hbar(np.std(df_testes_brightness.loc[inds,'Brightness Median']) + np.mean(df_testes_brightness.loc[inds,'Brightness Median']), height = 0, right=(bar_list[i]-whisker_width), left=(bar_list[i]+whisker_width), color = 'black')
    p.hbar(np.mean(df_testes_brightness.loc[inds,'Brightness Median']) - np.std(df_testes_brightness.loc[inds,'Brightness Median']), height = 0, right=(bar_list[i]-whisker_width), left=(bar_list[i]+whisker_width), color = 'black')
    p.segment(bar_list[i], np.mean(df_testes_brightness.loc[inds,'Brightness Median']), bar_list[i], np.std(df_testes_brightness.loc[inds,'Brightness Median']) + np.mean(df_testes_brightness.loc[inds,'Brightness Median']) , color = 'black')
    p.segment(bar_list[i], np.mean(df_testes_brightness.loc[inds,'Brightness Median']), bar_list[i], np.mean(df_testes_brightness.loc[inds,'Brightness Median']) - np.std(df_testes_brightness.loc[inds,'Brightness Median']), color = 'black')
    
    i += 1
    
p.plot_height = 350
p.plot_width = 350

p.xaxis.major_label_text_font_size = '12pt'
p.yaxis.major_label_text_font_size = '12pt'

p.xaxis.axis_label_text_font_size = '18pt'
p.yaxis.axis_label_text_font_size = '18pt'

bokeh.io.show(p)

In [ ]:
# Initial Difference of means
inds = df_testes_counts['Virus'] == 'AAV.CAP-A4'
CAPA4_testes_transduction = df_testes_counts.loc[inds, 'Cells Transduced per mm2']
CAPA4_testes_brightness = df_testes_brightness.loc[inds, 'Brightness Median']

inds = df_testes_counts['Virus'] == 'AAV9'
AAV9_testes_transduction = df_testes_counts.loc[inds, 'Cells Transduced per mm2']
AAV9_testes_brightness = df_testes_brightness.loc[inds, 'Brightness Median']

inds = df_testes_counts['Virus'] == 'AAV5'
AAV5_testes_transduction = df_testes_counts.loc[inds, 'Cells Transduced per mm2']
AAV5_testes_brightness = df_testes_brightness.loc[inds, 'Brightness Median']

three_sample_statistics(CAPA4_testes_transduction, 
                        AAV9_testes_transduction, 
                        AAV5_testes_transduction, 
                        labels = ['AAV-CAP.A4', 'AAV9', 'AAV5'], 
                        tissue = 'testes', 
                        measure = 'transduction (cells/mm\u00b2)')

three_sample_statistics(CAPA4_testes_brightness, 
                        AAV9_testes_brightness, 
                        AAV5_testes_brightness, 
                        labels = ['AAV-CAP.A4', 'AAV9', 'AAV5'], 
                        tissue = 'testes', 
                        measure = 'cell brightness')